# TRABALHO 03 - PARTE O1

## Modelagem de Dispersão

Autor: Mayara Dargas Sousa

Um contaminante emitido na atmosfera é transportado na direção do vento e dispersado por movimentos de ar perpendiculares ao vento, bem como por turbulência. A previsão da concentração dessa substância na área ao redor do ponto de emissão é um assunto de grande interesse em poluição atmosférica. No presente trabalho, faremos uso de um modelo gaussiano de dispersão que permite calcular as concentrações de um contaminante ao nível do solo. Considerando que o contaminante é emitido por uma chaminé localizada em um terreno plano. O modelo nos permite variar as condições meteorológicas (classe de estabilidade de acordo com as categorias definidas por Pasquill), a intensidade do vento e a temperatura (EPA, 2001). Portanto, serão contempladas as seguintes etapas: 

- Etapa 01: Implementar uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas.

- Etapa 02: Implementar a função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade.

- Etapa 03: Implementar a função de estimativa de sobrelevação da pluma utilizando os métodos de Davidson-Bryant, Holland e Briggs (quem fizer o método de Briggs ganha um ponto a mais). Considerando o efeito Tip-Downwash

- Etapa 04: Implementar a função do modelo gaussiano

- Etapa 05: Realizar simulações com o script criado, utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491.

- Etapa 06: Faça figuras e discuta os resultados.

## Etapa 01: Implementar função para determinar classes de estabilidade de Pasquill 

Utilizando o EZ-AERMOD foram importados dados de diferentes condições atmosférias e classificando conforme estação do ano. Posteriormente foi implementada a classificação de estabilidade de Pasqui conforme faixas de valores forncidadas pela universidade Federal do Espírito Santo.


In [169]:
import os
import pandas as pd

# Função para determinar a classe de estabilidade
def determinar_classe_estabilidade(row):
    u = row['u (m/s)']
    FCS = row['FCS (W/m2)']
    cco = row['cco']
    
    # Verificar se FCS é numérico
    if pd.isna(FCS) or not isinstance(FCS, (int, float)):
        return 'Unknown'  # ou qualquer valor padrão desejado
    
    FCS = float(FCS)  # Converte para float se não for
    
    if u < 2:
        if FCS > 700:
            return 'A'
        elif 350 <= FCS <= 700:
            return 'A - B'
        elif FCS < 350:
            return 'B'
    elif 2 <= u < 3:
        if FCS > 700:
            return 'A - B'
        elif 350 <= FCS <= 700:
            return 'B'
        elif FCS > 350:
            return 'C'
        if cco >= 4:
            return 'E'
        elif cco <= 3:
            return 'F'
    elif 3 <= u < 5:
        if FCS > 700:
            return 'B'
        elif 350 <= FCS <= 700:
            return 'B - C'
        elif FCS < 350:
            return 'C'
        if cco >= 4:
            return 'D'
        elif cco <= 3:
            return 'E'
    elif 5 <= u <= 6:
        if FCS > 700:
            return 'C'
        elif 350 <= FCS <= 700:
            return 'C - D'
        elif FCS < 350:
            return 'D'
        if cco >= 4:
            return 'D'
        elif cco <= 3:
            return 'D'
    elif u > 6:
        if FCS > 700:
            return 'C'
        elif 350 <= FCS <= 700:
            return 'D'
        elif FCS < 350:
            return 'D'
        if cco >= 4:
            return 'D'
        elif cco <= 3:
            return 'D'

### Testando implementação para diferentes condições atmosféricas 

In [170]:
# Pegando o caminho do diretório que estou
rootPath = os.getcwd()
print(rootPath)

# Definindo o diretório com os arquivos metar
meteoSFC = os.path.join(rootPath, 'inputs', 'meteoSFC')
print(meteoSFC)

# Listando os arquivos dentro do diretório
files = os.listdir(meteoSFC)
print(files)

# Nome das colunas que desejamos importar
columns = ['year', 'month', 'day', 'hour', 'FCS (W/m2)', 'u (m/s)', 'cco']

# Abrindo cada arquivo dentro da pasta e acumulando em uma lista chamada de metSFC
metSFC = []
for file in files:
    file_path = os.path.join(meteoSFC, file)
    df = pd.read_csv(file_path, usecols=columns)
    
    # Convertendo FCS (W/m2) para numérico
    df['FCS (W/m2)'] = pd.to_numeric(df['FCS (W/m2)'], errors='coerce')
    
    metSFC.append(df)

# Convertendo metSFC para DataFrame
metSFC = pd.concat(metSFC, ignore_index=True)

# Criando a coluna datetime
metSFC['datetime'] = pd.to_datetime(metSFC[['year', 'month', 'day', 'hour']])

# Aplicando a função para determinar a classe de estabilidade
metSFC['classe_estabilidade'] = metSFC.apply(determinar_classe_estabilidade, axis=1)

# Exibindo o DataFrame final
print(metSFC)


C:\ENS5173_MayaraDargas\ENS5173-2024.1_MayaraDargasSousa
C:\ENS5173_MayaraDargas\ENS5173-2024.1_MayaraDargasSousa\inputs\meteoSFC
['meteo.sfc.csv']
      year  month  day  hour  FCS (W/m2)  u (m/s)  cco            datetime  \
0     2023      1    1     1       -34.5      4.1    6 2023-01-01 01:00:00   
1     2023      1    1     2       -32.2      4.1    6 2023-01-01 02:00:00   
2     2023      1    1     3       -37.2      4.6    6 2023-01-01 03:00:00   
3     2023      1    1     4       -42.0      5.1    6 2023-01-01 04:00:00   
4     2023      1    1     5       -12.5      2.6    6 2023-01-01 05:00:00   
...    ...    ...  ...   ...         ...      ...  ...                 ...   
8757  2023     12   31    22        -9.2      2.1    6 2023-12-31 22:00:00   
8758  2023     12   31    23      -999.0    999.0    6 2023-12-31 23:00:00   
8759  2023     12   31    24      -999.0    999.0    6 2024-01-01 00:00:00   
8760  2023     12   31    24      -999.0    999.0    6 2024-01-01 00:00:

## Etapa 02: Implementar a função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade

In [171]:
import pandas as pd

# Função para calcular sigmaY e sigmaZ com base nos parâmetros
def sigmaYZ(x, classe, urbOrRural):
    if urbOrRural == 'urbano':
        if classe == 'A-B':
            sigmaY = 0.32 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.24 * x * (1 + 0.001 * x) ** 0.5
        elif classe == 'C':
            sigmaY = 0.22 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'D':
            sigmaY = 0.16 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.14 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe == 'E-F':
            sigmaY = 0.11 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0015 * x) ** (-0.5)
        else:
            raise ValueError('Classe de estabilidade errada')
    elif urbOrRural == 'rural':
        if classe == 'A':
            sigmaY = 0.22 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'B':
            sigmaY = 0.16 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.12 * x
        elif classe == 'C':
            sigmaY = 0.11 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0002 * x) ** (-0.5)
        elif classe == 'D':
            sigmaY = 0.08 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.06 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe == 'E':
            sigmaY = 0.06 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.03 * x * (1 + 0.0003 * x) ** (-1)
        elif classe == 'F':
            sigmaY = 0.04 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.016 * x * (1 + 0.0003 * x) ** (-1)
        else:
            raise ValueError('Classe de estabilidade errada')
    else:
        raise ValueError('Tipo de ambiente deve ser "urbano" ou "rural"')
    
    return sigmaY, sigmaZ

### Testando implementação para todas as classes de estabilidade

In [172]:

# Valores de x para calcular
x_values = [100, 1000, 10000]
environments = ['urbano', 'rural']
classes_urbano = ['A-B', 'C', 'D', 'E-F']
classes_rural = ['A', 'B', 'C', 'D', 'E', 'F']

# Lista para armazenar os resultados
results = []

# Calculando sigmaY e sigmaZ para cada combinação de x, classe e ambiente
for x in x_values:
    for environment in environments:
        if environment == 'urbano':
            classes = classes_urbano
        elif environment == 'rural':
            classes = classes_rural
        
        for classe in classes:
            sigmaY, sigmaZ = sigmaYZ(x, classe, environment)
            results.append({'x': x, 'classe': classe, 'environment': environment, 'sigmaY': sigmaY, 'sigmaZ': sigmaZ})

# Convertendo os resultados para um DataFrame
df_results = pd.DataFrame(results)

# Exibindo a tabela com os resultados
print(df_results)


        x classe environment       sigmaY       sigmaZ
0     100    A-B      urbano    31.378582    25.171412
1     100      C      urbano    21.572775    20.000000
2     100      D      urbano    15.689291    13.794610
3     100    E-F      urbano    10.786387     7.460038
4     100      A       rural    21.890818    20.000000
5     100      B       rural    15.920595    12.000000
6     100      C       rural    10.945409     7.921180
7     100      D       rural     7.960298     5.911976
8     100      E       rural     5.970223     2.912621
9     100      F       rural     3.980149     1.553398
10   1000    A-B      urbano   270.449362   339.411255
11   1000      C      urbano   185.933936   200.000000
12   1000      D      urbano   135.224681   122.788123
13   1000    E-F      urbano    92.966968    50.596443
14   1000      A       rural   209.761770   200.000000
15   1000      B       rural   152.554014   120.000000
16   1000      C       rural   104.880885    73.029674
17   1000 

## Etapa 03: Implementar a função de estimativa de sobrelevação da pluma

### Efeito Tip-Downwash

In [173]:
#determinar a tendência da pluma a ser dirigida em direção ao solo junto a chaminédef Holland(vs, d, u, p, Ts, Tamb):
def hg_tip_downwash(hg, d, vs, u):
    if vs >= 1.5 * u:
        hg2 = hg + (2 * d * ((vs / u) - 1.5))
    else:
        hg2 = hg 
    return hg2
#se vs é maior ou igual a 1.5 vezes u consederar hg2

### Método de Davidson-Bryant

In [174]:
# Função para estimar o deltaH com base na equação de Davidson-Bryant

def DavidsonBryant(d, vs, u, Ts, Tamb):
    deltaH_DavidsonBryant = (d*(vs/u)**(1.4))*(1+(Ts-Tamb)/Ts)
    return deltaH_DavidsonBryant

# d: diâmetro da chaminé (m)
# vs: velocidade do efluente na saída da chaminé (m/s)
# u: velocidade do vento a 10 metros (m/s)
# deltaT: temperatura do gás na chaminé menos a tempeatura ambiente (K)
# Ts: Tempetarura do gás na saída da chaminé (K)
# Tamb: Temperatura ambiente

### Método de Holland 

In [175]:
# Função para estimar o deltaH com base na equação de Holland

def Holland(vs, d, u, p, Ts, Tamb, classe):
    deltaH_Holland = (vs * d / u) * (1.5 + (2.68 * 10**-3) * (p * ((Ts - Tamb) / Ts) * d))

    if classe == 'A':
        deltaH_Holland *= 1.20  # aumenta 20%
    elif classe == 'B':
        deltaH_Holland *= 1.15  # aumenta 15%
    elif classe == 'C':
        deltaH_Holland *= 1.10  # aumenta 10%
    elif classe == 'E':
        deltaH_Holland *= 0.90  # diminui 10%
    elif classe == 'F':
        deltaH_Holland *= 0.80  # diminui 20%
    
    return deltaH_Holland

# Se condições atmosféricas instáveis fazer acréscimo de 10-20% de delta H em delta H (se classe igual a A acrescentar 20%, se classe iguaal a B acrestar 15% e se classe igual a C acrescentar 10%)
# Se forem estáveis, diminui-se igual quantidade (Se classe igual a E diminui 10% e se classe igual a F diminui 20%)

### Método de Briggs 

In [176]:
import math

# Constante pi
pi = math.pi  

# Função para calcular a vazão volumétrica (Qo)
def vazao_volumetrica(rc, vs):
    Qo = pi * (rc**2) * vs
    return Qo

# Função para calcular o parâmetro de flutuabilidade (Fb)
def flutuabilidade(Qo, Tamb, Ts):
    Fb = (9.8 / pi) * Qo * (1 - (Tamb / Ts))
    return Fb

# Função para calcular o gradiente de temperatura potencial
def grad_temp_potencial(G, classe):
    if classe == 'A':
        G = -0.020
    elif classe == 'B':
        G = -0.018
    elif classe == 'C':
        G = -0.016
    elif classe == 'D':
        G = -0.010
    elif classe == 'E':
        G = 0.005
    else:
        G = 0.028
    grad = G - (-0.0098)
    return grad

# Função para calcular o índice de estabilidade (S)
def indice_de_estabilidade(Tamb, d_theta, d_z):
    S = (9.8 / Tamb) * (d_theta / d_z)
    return S

# Função para calcular o fluxo momentâneo
def fluxo_momentaneo(vs, d, Tamb, Ts):
    Fm = (vs**2) * (d**2) * (Tamb / (4 * Ts))
    return Fm

# g: aceleração da gravidade (9,8 m²/s)
# deltaT: temperatura do gás na chaminé menos a tempeatura ambiente (K)
# Ts: Tempetarura do gás na saída da chaminé (K)
# Tamb: Temperatura ambiente
# rc: raio da chaminé (m)
# vs: velocidade do efluente na saída da chaminé (m/s)
# G: Valor médio do gradiente de temperatura ambiente


# Função principal de Briggs
def Briggs(Fb, vs, d, u, hg2, deltaT, deltaTc, S, classe, Ts):
    # Inicialização de H_Briggs com um valor padrão
    H_Briggs = hg2
    
    # Determinar se a atmosfera é instável ou estável com base na classe de Pasquill
    tipo_atmosfera = 'instavel' if classe in ['A', 'B', 'C', 'D'] else 'estavel'

    if tipo_atmosfera == 'instavel':
        if Fb < 55:
            deltaTc = 0.0297 * Ts * (vs**(1/3) / (d**(2/3)))
            if deltaTc > deltaT:
                H_Briggs = hg2 + 3 * d * (vs / u)
            else:
                H_Briggs = hg2 + (21.425 * ((Fb**(3/4)) / u))
        elif Fb >= 55:
            deltaTc = 0.00575 * Ts * (vs**(2/3) / (d**(1/3)))
            if deltaTc > deltaT:
                H_Briggs = hg2 + 3 * d * (vs / u)
            else:
                H_Briggs = hg2 + (2.6 * ((Fb / u)**(1/3)))
    
    if tipo_atmosfera == 'estavel':
        deltaTc = 0.01958 * Ts * vs * (S**(1/2))
        if deltaT < deltaTc:
            H_Briggs = hg2 + 3 * d * (vs / u)
        else:
            H_Briggs = hg2 + (2.6 * ((Fb**(3/4)) / u))
    
    return H_Briggs



### Testando implementação para todos os métodos 

In [177]:
# Exemplo de parâmetros de entrada
rc = 2.5  # raio da chaminé em metros
vs = 10  # velocidade do efluente em m/s
u = 5  # velocidade do vento a 10 metros em m/s
p = 1013.25  # pressão atmosférica em mb
Ts = 600  # temperatura do gás na saída da chaminé em K
Tamb = 300  # temperatura ambiente em K
classe = 'B'  # classe de estabilidade atmosférica

# Cálculo da vazão volumétrica (Qo)
Qo = vazao_volumetrica(rc, vs)

# Cálculo do parâmetro de flutuabilidade (Fb)
Fb = flutuabilidade(Qo, Tamb, Ts)

# Ajuste da altura da chaminé considerando downwash
hg = 100  # altura original da chaminé em metros
#hg2 = hg_tip_downwash(hg, rc*2, vs, u)  # rc*2 é o diâmetro

# Cálculo do deltaT (diferença de temperatura)
deltaT = Ts - Tamb

# Cálculo dos deltaHs
deltaH_Holland = Holland(vs, rc*2, u, p_mb, Ts, Tamb, classe)
deltaH_DavidsonBryant = DavidsonBryant(rc*2, vs, u, Ts, Tamb)
H_Briggs = Briggs(Fb, vs, rc*2, u, hg2, deltaT, 0, S, classe, Ts)

# Exibindo os resultados
print("H_Briggs:", H_Briggs)
print("deltaH_Holland:", deltaH_Holland)
print("deltaH_DavidsonBryant:", deltaH_DavidsonBryant)


print('  ')

#Cálculo da altura efetiva

print("A altura efetiva pelo método de Briggs é :", H_Briggs)

def hg_tip_downwash(hg, d, vs, u):
    if vs >= 1.5 * u:
        hg2 = hg + (2 * d * ((vs / u) - 1.5))
    else:
        hg2 = hg 
    return hg2

print("A altura efetiva pelo método de Holland é :", deltaH_Holland + hg2 )
print("A altura efetiva pelo método de DavidsonBryant é :", deltaH_DavidsonBryant + hg2 )

H_Briggs: 115.24885571203681
deltaH_Holland: 95.3209125
deltaH_DavidsonBryant: 19.792618661593412
  
A altura efetiva pelo método de Briggs é : 115.24885571203681
A altura efetiva pelo método de Holland é : 200.32091250000002
A altura efetiva pelo método de DavidsonBryant é : 124.79261866159341


## Etapa 04: Implementar a função do Modelo Gaussiano 

In [180]:
# Criando uma função do modelo gaussiano
import numpy as np
def modeloGaussiano(qs,sigmaY,sigmaZ,u,y,z,H):
    termo1 = qs/(2*np.pi*sigmaY*sigmaZ*u)
    termo2 = np.exp((-y**2)/(2*sigmaY**2))
    termo3 = np.exp((-(z-H)**2)/(2*sigmaZ**2)) + np.exp((-(z+H)**2)/(2*sigmaZ**2))
    conc = termo1*termo2*termo3
    conc = conc*10**6
    return conc

### Testando implementação com dados aleatórios

In [181]:
# Utilizando a função do modelo gaussiano
qs = 100 # g/s
sigmaY = 10 # m (tem que ser os dados que eu gerei antes)
sigmaZ = 15 # m
u = 10 # m/s
y = 0 # estimando na direção do vento
z = 1.5 # altura do nariz
H = 50 # m cálculad com algum dos métodos 

# Chamando a função do modelo gaussiano
conc = modeloGaussiano(qs,sigmaY,sigmaZ,u,y,z,H)

print(f"A concentração estimada é = {conc} [\u00b5g/m³]")

A concentração estimada é = 86.2052564132534 [µg/m³]


## Etapa 05: Realizar simulações com o script criado

Utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491.

Serão utilizadas as seguintes premissas:
- Classes, velocidade do vento: Pegar um dia de cada classe para cada estação do ano com os dados do AZERMOD da primeira etapa. 
- Altura da chaminé: testar em um primeiro momento 50, 100 e 150 metros; 
- Considerar taxa de amissão máxima do primeiro trabalho.

Resultados para o poluente 'PM':
- Emissão Média (g/s): 1000.3108197845369 
- Emissão Mínima (g/s): 996.5181218559003 
- Emissão Máxima (g/s): 1004.0695917785495 

Resultados para o poluente 'SO2':
- Emissão Média (g/s): 1871.7653708790822 
- Emissão Mínima (g/s): 1864.6685360707215 
- Emissão Máxima (g/s): 1878.7987240291943 

Resultados para o poluente 'NOX':
- Emissão Média (g/s): 106.39508423944257 
- Emissão Mínima (g/s): 105.99168520823049 
- Emissão Máxima (g/s): 106.79487483955421

In [183]:
# Criando domínio de modelagem 
x = np.linspace(-100,10000,500)
#print(x)
y = np.linspace(-10000,10000,500)

# Criando matrizes de x e y
xx,yy = np.meshgrid(x,y)
#print(xx.shape)


# Adotando inputs
classe = 'A'
urbOrRural = 'urbano' 
hg = 150 # m altura geométrica da chaminé
qs = 100 # g/s
sigmaY = 10 # m
sigmaZ = 15 # m
u = 5 # m/s
y = 0 # estimando na direção do vento
z = 1.5 # altura do nariz
d = 1 # em metros
vs = 10 # em m/s
Ts = 300 # em Kelvin
Tamb = 293 # em Kelvin

# Estimando o coeficiente de dispersão lateral e vertical
sigmaY,sigmaZ = sigmaXY(xx,classe,urbOrRural)
#print(sigmaY)

if vs>1.5*u:
    # Estimando a sobrelevação da pluma (deltaH)
    deltaH = deltaHdavidsonBryant(d,vs,u,Ts,Tamb)
    print('deltaH = '+str(deltaH))
    hef = hg+deltaH
    print('A pluma subiu')
else:
    hef = hg +2*d*((vs/u)-1.5)
    print('A pluma caiu')

# Utilizando a matriz de yy como input na função do modelo gaussiano
conc = modeloGaussiano(qs,sigmaY,sigmaZ,u,yy,z,hef)

# Visualização dos resultados no espaço
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
fig,ax = plt.subplots()
ax.contourf(xx,yy,conc+0.1,norm = LogNorm())

# Corte em y - sobre o eixo x
fig,ax = plt.subplots()
ax.plot(x,conc[250,:])

# Corte em x - sobre o eixo y 
fig,ax = plt.subplots()
ax.plot(x,conc[:,100])

NameError: name 'sigmaXY' is not defined